In [1]:
import sqlite3
import pandas as pd
from datetime import datetime

In [2]:
db_path = "results.sca"
conn = sqlite3.connect(db_path)

In [4]:
# Create a dataframe with just the runId and runNames
query = """
SELECT 
    runId,
    attrValue AS runName
FROM runAttr
WHERE attrName = 'configname'
"""

df = pd.read_sql_query(query, conn)

In [ ]:
# Add Host Count per run (Wireless)
query = """
SELECT 
    runId,
    paramValue AS wirelessHostCount
FROM runParam
WHERE paramKey = 'WiredAndWirelessHostsWithAP.num_wireless'
  AND paramOrder = 0
ORDER BY runId;
"""

temp = pd.read_sql_query(query, conn)
df = pd.merge(df, temp, on="runId", how="left")

In [ ]:
# Add Host Count per run Wired
query = """
SELECT runId, MAX(paramValue) AS wiredHostCount
FROM runParam
WHERE paramKey = 'WiredAndWirelessHostsWithAP.num_wired'
GROUP BY runId
ORDER BY runId;
"""

temp = pd.read_sql_query(query, conn)
df = pd.merge(df, temp, on="runId", how="left")

In [3]:
### PDR ###

# Query to calculate sent and received packets per run
query = """
WITH ReceivedPackets AS (
        SELECT r.runId, s.scalarvalue AS packetReceived
        FROM scalar s JOIN run r ON r.runId = s.runId
        WHERE s.scalarName = 'packetReceived:count' 
        AND s.moduleName = 'WiredAndWirelessHostsWithAP.sinkNode.udp'
    ),
    SentPackets AS (
        SELECT r.runId, SUM(s.scalarvalue) AS packetSent
        FROM scalar s JOIN run r ON r.runId = s.runId
        WHERE s.scalarName = 'packetSent:count' 
        AND s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].udp'
        GROUP BY r.runId
    )
    SELECT 
        r.runId,
        COALESCE(sp.packetSent, 0) AS packetsSent,
        COALESCE(rp.packetReceived, 0) AS packetsReceived
    FROM run r
    LEFT JOIN SentPackets sp ON r.runId = sp.runId
    LEFT JOIN ReceivedPackets rp ON r.runId = rp.runId
    ORDER BY r.runId
"""

temp = pd.read_sql_query(query, conn)


In [5]:
df = pd.merge(df, temp, on="runId", how="left")

,runId,packetsSent,packetsReceived,runName
0,1,15000.0,9124.0,T1_A_Hosts_25
1,2,6000.0,5990.0,T1_A_Hosts_10
2,3,9000.0,8984.0,T1_A_Hosts_15
3,4,12000.0,9324.0,T1_A_Hosts_20
4,5,30000.0,8583.0,T1_A_Hosts_50


In [6]:
# Packet Dropped
query = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS packetDropped
FROM scalar s
JOIN run r ON r.runId=s.runId
WHERE s.scalarName = 'packetDrop:count'
AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac'
  )
GROUP BY r.runId
"""

temp = pd.read_sql_query(query, conn)
temp.head()

,runId,packetDropped
0,1,542144.0
1,2,114159.0
2,3,278252.0
3,4,416668.0
4,5,1095058.0


In [7]:
df = pd.merge(df, temp, on="runId", how="left")

In [8]:
df.head()

,runId,packetsSent,packetsReceived,runName,packetDropped
0,1,15000.0,9124.0,T1_A_Hosts_25,542144.0
1,2,6000.0,5990.0,T1_A_Hosts_10,114159.0
2,3,9000.0,8984.0,T1_A_Hosts_15,278252.0
3,4,12000.0,9324.0,T1_A_Hosts_20,416668.0
4,5,30000.0,8583.0,T1_A_Hosts_50,1095058.0


In [9]:
# Add simulation time
df["simTime"] = 60
df.loc[df["runName"].str.contains("T3|T4", na=False), "simTime"] = 30

In [10]:
# Mac Frames Transmitted
# Packet Sent to Lower Mac Layer 
query = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS macFrameSent
FROM scalar s
JOIN run r ON r.runId=s.runId
WHERE s.scalarName = 'packetSentToLower:count'
AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac'
  )
GROUP BY r.runId
"""

temp = pd.read_sql_query(query, conn)
temp.head()


,runId,macFrameSent
0,1,28031.0
1,2,19950.0
2,3,29680.0
3,4,26855.0
4,5,29526.0


In [11]:
df = pd.merge(df, temp, on="runId", how="left")


In [12]:
# Mac frames Retransmitted

query = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS macFrameRetransmitted
FROM scalar s
JOIN run r ON r.runId=s.runId
WHERE s.scalarName = 'packetSentToPeerWithRetry:count' AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac.dcf'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac.dcf'
  )
GROUP BY r.runId
"""

temp = pd.read_sql_query(query, conn)
temp.head()

,runId,macFrameRetransmitted
0,1,7840.0
1,2,7170.0
2,3,10803.0
3,4,7125.0
4,5,10158.0


In [13]:
df = pd.merge(df, temp, on="runId", how="left")


In [14]:
df.head()

,runId,packetsSent,packetsReceived,runName,packetDropped,simTime,macFrameSent,macFrameRetransmitted
0,1,15000.0,9124.0,T1_A_Hosts_25,542144.0,60,28031.0,7840.0
1,2,6000.0,5990.0,T1_A_Hosts_10,114159.0,60,19950.0,7170.0
2,3,9000.0,8984.0,T1_A_Hosts_15,278252.0,60,29680.0,10803.0
3,4,12000.0,9324.0,T1_A_Hosts_20,416668.0,60,26855.0,7125.0
4,5,30000.0,8583.0,T1_A_Hosts_50,1095058.0,60,29526.0,10158.0


In [ ]:
# Packet Dropped due to Queue Overflow

query = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS packetDroppedQueueOverflow
FROM scalar s
JOIN run r ON r.runId = s.runId
WHERE s.scalarName = 'packetDropQueueOverflow:count'
  AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac'
  )
GROUP BY r.runId;
"""

temp = pd.read_sql_query(query, conn)
temp.head()

In [ ]:
df = pd.merge(df, temp[["runId", "packetDroppedQueueOverflow"]], on="runId", how="left")

In [ ]:
df.head()

In [ ]:
# --- Approximate BER Calculation ---

# Numerator: total bits in incorrectly received packets
query_ber_num = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS erroneousBits
FROM scalar s
JOIN run r ON r.runId = s.runId
WHERE s.scalarName = 'packetDropIncorrectlyReceived:sum(packetBytes)'
  AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac'
  )
GROUP BY r.runId
"""

df_ber_num = pd.read_sql_query(query_ber_num, conn)
# Multiply by 8 to convert bytes → bits
df_ber_num["erroneousBits"] = df_ber_num["erroneousBits"] * 8

# Denominator: total bits transmitted
query_ber_den = """
SELECT 
    r.runId,
    SUM(s.scalarvalue) AS totalBytesSent
FROM scalar s
JOIN run r ON r.runId = s.runId
WHERE s.scalarName = 'packetSentToLower:sum(packetBytes)'
  AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].mac'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].mac'
  )
GROUP BY r.runId
"""

df_ber_den = pd.read_sql_query(query_ber_den, conn)
df_ber_den["totalBitsSent"] = df_ber_den["totalBytesSent"] * 8

In [ ]:
# Merge numerator and denominator
df = pd.merge(df, df_ber_num[["runId", "erroneousBits"]], on="runId", how="left")
df = pd.merge(df, df_ber_den[["runId", "totalBitsSent"]], on="runId", how="left")


df.head()

In [ ]:
# Collision Count

query = """
SELECT 
    r.runId,
    SUM(s.scalarValue) AS collisionCount
FROM scalar s
JOIN run r ON r.runId = s.runId
WHERE s.scalarName = 'collisionCount'
  AND (
        s.moduleName LIKE 'WiredAndWirelessHostsWithAP.wirelessHost[%].wlan[%].radio'
     OR s.moduleName LIKE 'WiredAndWirelessHostsWithAP.accessPoint.wlan[%].radio'
  )
GROUP BY r.runId;
"""

temp = pd.read_sql_query(query, conn)
temp.head()

In [ ]:
df = pd.merge(df, temp[["runId", "collisionCount"]], on="runId", how="left")
df.head()

In [ ]:
# Packet Drop Ratio
df["PDR"] = df["packetsReceived"] / df["packetsSent"]
df["PDR(%)"] = df["PDR"]*100

# MAC Retransmission Rate
df["MAC_RetryRate"] = df["macFrameRetransmitted"] / df["macFrameSent"]
df["MAC_RetryRate"] = df["MAC_RetryRate"].fillna(0)

# Calculate Queue Overflow Rate
df["QueueOverflowRate"] = df["packetDroppedQueueOverflow"] / (df["macFrameSent"] + df["packetDroppedQueueOverflow"])
df["QueueOverflowRate"] = df["QueueOverflowRate"].fillna(0)

# Calculate BER
df["BER"] = df["erroneousBits"] / df["totalBitsSent"]
df["BER"] = df["BER"].fillna(0)


In [ ]:
# End to End Delay

query = """
SELECT 
    runId,
    statMin AS minEndToEndDelay,
    statMax AS maxEndToEndDelay,
    statMin As meanEndToEndDelay,
    statStdDev AS stdDevEndToEndDelay
FROM statistic
WHERE statName = 'endToEndDelay:histogram'AND moduleName LIKE 'WiredAndWirelessHostsWithAP.sinkNode.app[%]'
"""

temp = pd.read_sql_query(query, conn)
temp.head()

In [ ]:
df = pd.merge(df, temp[["runId", "minEndToEndDelay"]], on="runId", how="left")
df = pd.merge(df, temp[["runId", "maxEndToEndDelay"]], on="runId", how="left")
df = pd.merge(df, temp[["runId", "meanEndToEndDelay"]], on="runId", how="left")
df = pd.merge(df, temp[["runId", "stdDevEndToEndDelay"]], on="runId", how="left")
df.head()

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8,5))
sns.lineplot(data=df, x="# of Hosts", y="PDR(%)", marker='o')
plt.title("Packet Delivery Ratio vs Number of Hosts")
plt.xlabel("Number of Hosts")
plt.ylabel("PDR (%)")
plt.grid(True)
plt.show()